# Computation on Arrays - Universal Functions

We should now feel relatively comfortable creating and array and then indexing and slicing it as we need.

Next, we turn to doing computation on arrays. NumPy implements most of the standard computations we need to perform on arrays using *universal functions,* or ufuncs for short. NumPy's ufuncs are optimized using vectorized operations so are in general extremely fast - and we should always try and use them rather than writing our own method.

Let's get started exploring the world of ufuncs by loading NumPy into our notebook

In [ ]:
import numpy as np
np.random.seed(1234567890)

## Slowness of loops

Python's default implementation of some operations is quite slow, mostly due to the fact that its a dynamic, interpreted language. This means that types are flexible so at each operation a series of type checks and other things need to be done, implying operations are generally not compiled to efficient machine code.

This manifests itself when many small operations have to be repeated: like in a loop that would do the same thing to each element of an array:

In [ ]:
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

In [ ]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

The bottleneck here is the type checking and function dispatches that the Python engine is performing at each cycle of the loop: each time it examines the objects type, and looks for the correct function to use on that type.

## Introducting UFuncs

NumPy provides a convenient interface into a statically typed, compiled routine. This can be accomplished by simply performing an operation on the array, which will then be applied to each element, and recall we know that each element of a numpy array must be of the same time; so we save time on all the type checking related slow downs.

Here's an example using a NumPy ufunc to perform the same operation as above:

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

In [ ]:
%timeit (1.0 / big_array)

Vectorized operations in NumPy are implemented via ufuncs, which offer the ability to quickly execute repeated operations on values in NumPy arrays. 

Ufuncs are extremely flexible – we can also operate between two arrays:

In [ ]:
np.arange(5) / np.arange(1, 6)

And multidimensional arrays:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

## Numpy's UFuncs

Numpy's ufuncs make use of Python's native arithmetic operators, so are easy to use:

In [ ]:
x = np.arange(10)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 5)
print("x / 2 =", x / 5)
print("x // 2 =", x // 5)  # floor division

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 5)
print("x % 2  = ", x % 5)

Ufuncs can be strung together as in any combination, and use the standard order of operations to execute:

In [ ]:
-(0.5*x + 1) ** 2


Each of these arithmetic operators are wrappers around specific NumPy functions, for example:

In [ ]:
all(np.add(x, 2) == x + 2)

This table relates the arithmetic operator to the NumPy function:


| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|

Source: Jake VanderPlas (2016), Python Data Science Handbook Essential Tools for Working with Data, O'Reilly Media.

### Absolute Value

NumPy also plays well with Python's native absolute value function

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

### Exponents and Logarithms

We often work with exponentials and logarithms:

In [ ]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

NumPy has specialized versions that are useful for maintaining precision with small valued inputs:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

There's also a bunch of trigonometric functions, but these are less common in our work so we skip over them.

### Some useful UFuncs are living in Scipy

The SciPy module (that we discuss further later) has some additional ufuncs that are more specialized in nature. (This may be suggested by how we import them!)

Here' some useful ones for stats:

In [ ]:
from scipy import special

In [ ]:
x = [1, 5, 10]

In [ ]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

## 'Advanced' UFunc Features

### Specifying output

We are used to seeing

```python
y = np.multiply(x, 10)
```

But we can achive the same result using the ``out`` argument, which is available in all ufuncs - provided we the location of the out argument exists beforehand.

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

So why might we be interested in this feature?

Notice that:

In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

is equivalent to : `y[::2] = 2 ** x`.

If we perfom this operation on a large array:

In [ ]:
big_array = np.random.randint(1, 100, size=50000000)
y = np.zeros(2 * big_array.size)

In [ ]:
%timeit np.power(2, big_array, out=y[::2])

In [ ]:
%timeit y[::2] = 2 ** big_array

There is large difference in the timing.

What's the difference? The latter executes 2 operations: First, it creates of a temporary array to hold the results of `2 ** x`. Second it copies those values into the y array.

### Aggregation

Some aggregates can be computed directly from an array when we want to reduce it using a particular operation: 

In [ ]:
x = np.arange(1, 6)
np.add.reduce(x)

In [ ]:
np.multiply.reduce(x)

In [ ]:
np.add.accumulate(x)


In [ ]:
np.multiply.accumulate(x)


### Outer products

In [ ]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

## Challenge

Suppose we have two firms in market operating under Cournot compeition.
Let $q1$ and $q2$ denote the quantities produced by each firm respectively, and $Q=q1+q2$.

Market demand can be either linear:

$$
P(Q) = a \times Q + b
$$

or be isoelastic:

$$
P(Q) = k Q^ {-\epsilon}
$$

Assume that marginal cost of production is $c$ for both firms, and they can only produce integer valued outputs in the range $q_i \in [0,10]$.

1. Write a function that returns the market price for the linear demand function for all possible market quantities. Return the price function as a matrix that can be indexed so that price[5,5] yields the market price when both firms produce 5 units of output

2. Repeat 1. for the isoelastic case.

3. Write a function that returns the profit for firm 1, for any combination of inputs for the either demand model. 
    1. What is his profit when they both produce 5 units of output, when a = -3, b = 30 and c=1?
    2. What is his profit when they both produce 5 units of output, when k = -1, $\epsilon$ = -3 and c=1?

In [ ]:
def price_lin(a,b):
    q1 = np.arange(0, 11)
    q2 = np.arange(0, 11)
    Q = np.add.outer(q1, q2)
    p = a * Q + b
    return p

def price_iso(k,e):
    q1 = np.arange(0, 11)
    q2 = np.arange(0, 11)
    Q = np.add.outer(q1, q2)
    p = k * Q ** (-e)
    return p

def profit_firm1(q1, q2, parm1, parm2, c = 1, price_fct = price_lin):
    '''
    Inputs:
    - q1, q2: quantities
    - parm1, parm2: price function parameters
    - c: unit costs, default is 1
    - price_fct: price function, either "price_lin" (default) or "price_iso"
    - - if "linear": parm1 = a, parm2 = b
    - - if "isoelastic": parm1 = k, parm2 = e
    '''
    price = price_fct(parm1,parm2)[q1,q2]
    profit = (price - c)*q1
    return profit

print("This is the linear price matrix:")
print(price_lin(-3,30))
print("")
print("This is the isoelastic price matrix:")
print(price_iso(-1,-3))
print("")
print("This is firm 1's profit:")
print(profit_firm1(5,5,-1,-3, price_fct = price_lin))

## Additional Info on ufuncs:

See https://docs.scipy.org/doc/numpy/reference/ufuncs.html for more ufuncs inside NumPy, and https://docs.scipy.org/doc/scipy/reference/special.html for ufuncs in SciPy's `special` module.